In [3]:
import pandas as pd
import numpy as np


In [4]:
data = pd.read_csv('./datasets/ACCIDENTS_GU_BCN_2010.csv', encoding='latin-1')

In [6]:
data['Date'] = '2010' + '-' + data[u'Dia de mes'].apply(lambda x: str(x)) + '-' + data[u'Mes de any'].apply(lambda x: str(x)) 
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
counts2010 = data['Date'].value_counts()
print('2010 mean:', counts2010.mean())

data2 = pd.read_csv('./datasets/ACCIDENTS_GU_BCN_2013.csv', encoding='latin-1')
data2['Date'] = '2013' + '-' + data2[u'Dia de mes'].apply(lambda x: str(x)) + '-' + data2[u'Mes de any'].apply(lambda x: str(x)) 
data2['Date'] = pd.to_datetime(data2['Date'], dayfirst=True)
counts2013 = data2['Date'].value_counts()
print('2013 mean:', counts2013.mean())



2010 mean: 24.81095890410959
2013 mean: 25.90958904109589


It seems to suggest that the mean for 2013 is higher. But is it statistically significant?

We can calculate the 95% confidence interval as follows:

In [10]:
n = len(counts2013)

mean = counts2013.mean()
s = counts2013.std()

ci = [mean - s*1.96/np.sqrt(n), mean + s*1.96/np.sqrt(n)]
print('2010 rate estimate:', counts2010.mean())
print('2013 rate estimate:', counts2013.mean())
print('CI for 2013: ', ci)

2010 rate estimate: 24.81095890410959
2013 rate estimate: 25.90958904109589
CI for 2013:  [24.975156065800284, 26.8440220163915]


#### Because the 2010 accident rate estimate does not fall in the range of plausible values of 2013, we say alternative hypothesis cannot be discarded. We say that it can't be ruled out that the mean rate of accident in 2013 was higher than in 2010. 

## P-value

In [11]:
m = len(counts2010)
n = len(counts2013)

p = (counts2013.mean() - counts2010.mean())

print("mean difference", p)


mean difference 1.0986301369863014


In [53]:
x = counts2010
y = counts2013
pool = np.concatenate([x, y]) # daily accidents for both years, pooled
np.random.shuffle(pool)

import random

N = 10000
diff = [0]*N
for i in range(N):
    p1 = [random.choice(pool) for _ in range(n)] # select 365 random values from pool
    p2 = [random.choice(pool) for _ in range(n)] # select 365 random values from pool
    diff[i] = (np.mean(p1) - np.mean(p2)) # an array of difference between both means, 10k in length
    #^ what exactly is going on in this step above? p1 and p2 are 2 random samples. you are finding out whether there's any difference in means between 10k pairs of samples. 
'''
diff:
[0.0027397260274000246,
 -0.29315068493150775,
 0.2136986301369852,
 -0.6054794520547944,
 0.19178082191780632,...]
'''

diff2 = np.array(diff)
w1 = np.where(diff2 > p)[0] # returns index of those with diff more than p
print(w1)
print(len(w1))
# Here, p value is num of w1 over total N. What is w1? -> first, diff2 contains those difference in means of 10k sample pairs. w1 is a filter to find those with differences larger than p. w1 is thus the proportion of those with differences larger than p, versus those less than P. 

'''
w1/N -> represents the probability of observing a difference as extreme as p (or more extreme) if the null hypothesis were true (i.e., there is no actual difference between the means of the two groups).

For instance, if w1 only has 1/10k, that shows that in fact, observing a difference is VERY uncommon, which means the actual observation of p, is less likely to have happened due to chance. 

w1 calculates how LIKELY it is to observe a difference more than p. if everyday we observe p (w1 being very long), say 50% chance, then we say that we are not very confident that an observation of p is due to chance. However, 
'''
print('p-value (Simulation) = ', len(w1)/float(N), 'Difference = ', p) #

if (len(w1)/float(N) < 0.05):
    print("The effect is likely")
else: 
    print("The effect is not likely")

[  28   39   62   72   82  123  136  163  195  204  226  240  257  263
  278  283  297  299  305  322  330  341  352  353  367  377  382  383
  391  431  440  476  540  551  552  595  604  631  641  681  685  708
  714  760  775  834  890  947  987  994 1075 1093 1108 1123 1161 1180
 1194 1201 1245 1252 1271 1275 1276 1284 1346 1356 1357 1360 1370 1377
 1394 1412 1441 1459 1499 1512 1520 1532 1568 1595 1622 1625 1636 1720
 1756 1767 1769 1785 1794 1846 1890 1897 1907 1928 1936 2012 2046 2078
 2107 2158 2170 2200 2218 2219 2229 2242 2243 2268 2285 2287 2297 2317
 2322 2350 2352 2367 2420 2435 2465 2500 2524 2541 2544 2546 2592 2622
 2628 2649 2697 2724 2732 2751 2753 2760 2826 2846 2858 2874 2893 2894
 2918 2940 2941 2961 2994 3001 3015 3059 3066 3072 3084 3099 3100 3103
 3119 3121 3146 3161 3183 3252 3259 3263 3267 3275 3301 3312 3323 3326
 3359 3378 3379 3384 3389 3426 3478 3554 3556 3571 3589 3592 3598 3603
 3614 3617 3645 3670 3681 3695 3703 3710 3752 3776 3778 3794 3799 3818
 3839 

In [51]:
(-2 -1)

-3